In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
import os
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.signal import welch 
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np


In [40]:
from scipy.signal import welch
import numpy as np

def compute_bandpower(data, fs, band):
    freqs, psd = welch(data, fs=fs, nperseg=fs//2)
    idx_band = np.logical_and(freqs >= band[0], freqs <= band[1])
    return np.trapz(psd[idx_band], freqs[idx_band])

def compute_rms(x):
    return np.sqrt(np.mean(np.square(x)))

def compute_hjorth_params(x):
    first_deriv = np.diff(x)
    second_deriv = np.diff(first_deriv)

    var_zero = np.var(x)
    var_d1 = np.var(first_deriv)
    var_d2 = np.var(second_deriv)

    mobility = np.sqrt(var_d1 / var_zero) if var_zero != 0 else 0
    complexity = np.sqrt(var_d2 / var_d1) if var_d1 != 0 else 0

    return mobility, complexity

In [41]:
class EEGWindowDataset(Dataset):
    def __init__(self, combined_data_path, training_data_path, sampling_rate=250, window_sec=2):
        self.combined_data = pd.read_csv(combined_data_path).values.astype(np.float32)
        self.training_data = pd.read_csv(training_data_path).values.astype(np.int64)

        self.window_size = int(window_sec * sampling_rate)
        self.half_window = self.window_size // 2

        self.total_rows = self.combined_data.shape[0]
        self.sampling_rate = sampling_rate

    def __len__(self):
        return len(self.training_data)

    def __getitem__(self, idx):
        class_label, center_idx = self.training_data[idx]
    
        # Define start and end of window
        start_idx = center_idx - self.half_window
        end_idx = center_idx + self.half_window

        # Edge handling (pad with zeros if window goes out of bounds)
        if start_idx < 0 or end_idx > self.total_rows:
            window = np.zeros((self.window_size, 4), dtype=np.float32)
            actual_start = max(0, start_idx)
            actual_end = min(self.total_rows, end_idx)
            window_offset_start = max(0, -start_idx)
            window[window_offset_start:window_offset_start + (actual_end - actual_start)] = \
                self.combined_data[actual_start:actual_end, :4]
        else:
            window = self.combined_data[start_idx:end_idx, :4]
    
        features = []
        for ch in range(4):  # 4 EEG channels
            signal = window[:, ch]
            alpha = compute_bandpower(signal, fs=self.sampling_rate, band=(8, 13))
            beta = compute_bandpower(signal, fs=self.sampling_rate, band=(13, 30))
            rms = compute_rms(signal)
            mobility, complexity = compute_hjorth_params(signal)
            features.extend([alpha, beta, rms, mobility, complexity])
    
        feature_tensor = torch.tensor(features, dtype=torch.float32)
        return feature_tensor, class_label

In [42]:
training_dataset = EEGWindowDataset("TrainingData/combined_data.csv", "TrainingData/training_data.csv", sampling_rate=250, window_sec=2)
testing_dataset = EEGWindowDataset("TrainingData/combined_data.csv", "TrainingData/testing_data.csv", sampling_rate=250, window_sec=2)
train_loader = DataLoader(training_dataset, batch_size=64, shuffle=False)
val_loader = DataLoader(testing_dataset, batch_size=64, shuffle=False)

# Model

In [43]:
# Define the MLP model
class EEGClassifier(nn.Module):
    def __init__(self):
        super(EEGClassifier, self).__init__()
        self.fc1 = nn.Linear(20, 32)       # Input: 20 features → Hidden: 32 neurons
        self.fc2 = nn.Linear(32, 11)       # Hidden: 32 neurons → Output: 11 classes

    def forward(self, x):
        x = F.relu(self.fc1(x))            # Activation
        x = F.softmax(self.fc2(x), dim=1)  # Softmax for classification
        return x

# Instantiate the model
model = EEGClassifier()

# Optimizer and loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()



In [49]:
def train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=30, device='cpu'):
    model.to(device)
    count = 0
    for epoch in range(num_epochs):
        # -------- Training Phase --------
        model.train()
        total_loss = 0

        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        

        # -------- Validation Phase --------
        if count == num_epochs - 1:
            model.eval()
            y_true, y_pred = [], []
    
            with torch.no_grad():
                for val_X, val_y in val_loader:
                    val_X, val_y = val_X.to(device), val_y.to(device)
                    outputs = model(val_X)
                    preds = torch.argmax(outputs, dim=1)
    
                    y_true.extend(val_y.cpu().numpy())
                    y_pred.extend(preds.cpu().numpy())
    
            # ----- Confusion Matrix -----
            cm = confusion_matrix(y_true, y_pred)
            print("Confusion Matrix:")
            print(cm)
    
            print("\nClassification Report:")
            print(classification_report(y_true, y_pred, digits=3))
            print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {avg_loss:.4f}")
        count += 1

In [50]:
train_model(model, train_loader, val_loader, optimizer, nn.CrossEntropyLoss(), num_epochs=30)



Epoch [1/30] - Loss: 2.1503
Epoch [2/30] - Loss: 2.1492
Epoch [3/30] - Loss: 2.1470
Epoch [4/30] - Loss: 2.1462
Epoch [5/30] - Loss: 2.1452
Epoch [6/30] - Loss: 2.1444
Epoch [7/30] - Loss: 2.1437
Epoch [8/30] - Loss: 2.1432
Epoch [9/30] - Loss: 2.1426
Epoch [10/30] - Loss: 2.1396
Epoch [11/30] - Loss: 2.1390
Epoch [12/30] - Loss: 2.1386
Epoch [13/30] - Loss: 2.1381
Epoch [14/30] - Loss: 2.1376
Epoch [15/30] - Loss: 2.1372
Epoch [16/30] - Loss: 2.1369
Epoch [17/30] - Loss: 2.1366
Epoch [18/30] - Loss: 2.1362
Epoch [19/30] - Loss: 2.1358
Epoch [20/30] - Loss: 2.1354
Epoch [21/30] - Loss: 2.1350
Epoch [22/30] - Loss: 2.1346
Epoch [23/30] - Loss: 2.1344
Epoch [24/30] - Loss: 2.1336
Epoch [25/30] - Loss: 2.1333
Epoch [26/30] - Loss: 2.1328
Epoch [27/30] - Loss: 2.1312
Epoch [28/30] - Loss: 2.1304
Epoch [29/30] - Loss: 2.1305
Epoch [30/30] - Loss: 2.1296
Confusion Matrix:
[[36  0  0  0  1  0  0  1  0  0  0]
 [ 2  0  0  0  1  0  0  3  3  3  0]
 [ 0  0  0  0  4  0  0  2  1  2  0]
 [ 0  0  0  0

C:\Users\shawn_hx71ej8\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\shawn_hx71ej8\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\shawn_hx71ej8\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.